<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0112Train_a_language_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- original: https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/language_modeling_from_scratch.ipynb
- date: 2022_0112
- filename: `2022_0112language_modeling_from_scratch.ipynb`

# 言語モデルの訓練
<!-- # Train a language model -->

<!-- colabでこのノートを開く場合、おそらく「Transformers」と「🤗 Datasets」をインストールする必要があるでしょう。次のセルのコメントを外して実行します。 -->
<!-- If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it. -->

In [1]:
import platform
isColab = platform.system() == 'Linux'
if isColab:
    !pip install datasets transformers > /dev/null 2>&1

このノートブックをローカルで開いている場合は，お使いの環境にこれらのライブラリの最新バージョンがインストールされていることを確認してください。
<!-- If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.-->

自分のモデルをコミュニティと共有し，推論 API を介して下図のような結果を生成するためには，さらにいくつかのステップを踏む必要があります。
<!-- To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow. -->

まず Hugging Face のウェブサイトから認証トークンを保存し (まだ登録していない場合は [こちら](https://huggingface.co/join))，次のセルを実行してユーザ名とパスワードを入力します。
<!-- First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password: -->

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

次に、Git-LFSをインストールする必要があります。
以下の説明をアンコメントしてください。
<!-- Then you need to install Git-LFS. Uncomment the following instructions: -->

In [3]:
if isColab:
    !apt install git-lfs > /dev/null 2>&1

トランスフォーマーのバージョンが 4.11.0 以上であることを確認してください (この機能は4.11.0で導入されました)。
<!-- Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version: -->

In [ ]:
import transformers
print(transformers.__version__)

複数の GPU や TPU を使って分散型でモデルを微調整するため，このノートのスクリプト版が [こちら](https://github.com/huggingface/transformers/tree/master/examples/language-modeling) にあります。
<!-- You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling). -->

# 言語モデルの訓練
<!-- # Train a language model -->

このノートブックでは，言語モデリング課題で [🤗 Transformers](https://github.com/huggingface/transformers) モデルを訓練する方法を見ていきます。ここでは，以下の 2 種類の言語モデリング課題を取り上げます。
<!-- In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover two types of language modeling tasks which are:-->

- 因果関係のある言語モデリング: モデルは文中の次のトークンを予測しなければなりません (そのため，ラベルは入力を右にシフトしたものと同じになります)。
モデルが不正をしないように，文中のトークン i+1 を予測しようとするときに，トークン i 以降のトークンにアクセスできないようにする注意マスクを取得します。

<!-- - Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence. -->

<center>
<img src="https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1" width="66%"><br/>
因果関係のある言語モデリング課題を表すウィジェット推論
</center>

<!-- ![因果関係のある言語モデリング課題を表すウィジェット推論](https://github.com/huggingface/notebooks/blob/master/examples/images/causal_language_modeling.png?raw=1) -->
<!-- ![Widget inference representing the causal language modeling task](images/causal_language_modeling.png) -->

- マスク化言語モデリング: モデルは，入力の文中でマスクされたトークンを予測しなければならない。
文全体にアクセスできるので，マスク化トークンの前後のトークンを使ってその値を予測することができます。

<!-- - Masked language modeling: the model has to predict some tokens that are masked in the input. 
It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value. -->

<center>
<img src="https://github.com/huggingface/notebooks/blob/master/examples/images/masked_language_modeling.png?raw=1" width="66%"><br/>
マスク化言語モデル課題を表現するウィジェット推論
<!-- Widget inference representing the masked language modeling task -->
</center>
<!-- ![widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/master/examples/images/masked_language_modeling.png?raw=1) -->

ここでは，それぞれの課題のデータセットを簡単に読み込んで前処理する方法と，`Trainer` API を使ってモデルを訓練する方法を見ていきます。
<!-- We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to train a model on it. -->

このノートでは、使用するコーパスでトークン化器を訓練していることを前提としています。
[トークン化器の訓練方法 (How to train a tokenizer)](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) ノートブック（[open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)）を参照してください。
<!-- This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook ([open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)). -->

このノートブックを分散環境やTPU上で直接実行できるスクリプトバージョンは [examples folder](https://github.com/huggingface/transformers/tree/master/examples) にあります。
<!-- A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples). -->

## データセットの準備
<!-- ## Preparing the dataset -->

それぞれの課題について [Wikitext 2]() データセットを例に挙げて説明します。
このデータセットは 🤗 Datasets ライブラリでとても簡単に読み込むことができます。
<!-- For each of those tasks, we will use the [Wikitext 2]() dataset as an example. 
You can load it very easily with the 🤗 Datasets library. -->

In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

上のデータセットを [the hub](https://huggingface.co/datasets) でホストされている任意のデータセットに置き換えることも，自分のファイルを使うこともできます。
以下のセルをアンコメントし，パスをあなたのファイルにつながる値に置き換えてください。
<!-- You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. 
Just uncomment the following cell and replace the paths with values that will lead to your files: -->

In [6]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

csv や JSON ファイルからデータセットを読み込むこともできます。
詳しくは [マニュアル，ローカルファイルからの読み込み](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) をご覧ください。
<!-- You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information. -->

実際の要素にアクセスするには，まずスプリットを選択し，次にインデックスを与える必要があります。
<!-- To access an actual element, you need to select a split first, then give an index: -->

In [ ]:
datasets["train"][10]

データがどのようなものかを知るために，次の関数では，データセットの中からランダムに選ばれたいくつかの例を表示します。
<!-- To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset. -->

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,text
0,"Jordan was recruited by numerous college basketball programs , including Duke , North Carolina , South Carolina , Syracuse , and Virginia . In 1981 , Jordan accepted a basketball scholarship to North Carolina , where he majored in cultural geography . \n"
1,= = Marriage and family = = \n
2,"Aretusa was a torpedo cruiser of the Partenope class built for the Italian Regia Marina ( Royal Navy ) in the 1880s . Laid down in June 1889 at the Cantiere navale fratelli Orlando shipyard , she was launched in March 1891 and was commissioned in September 1892 . Her main armament were her six torpedo tubes , which were supported by a battery of ten small @-@ caliber guns . Aretusa spent most of her career in the main Italian fleet , where she was primarily occupied with training exercises . At the start of the Italo @-@ Turkish War in September 1911 , she was assigned to the Red Sea Squadron in Italian Eritrea . She bombarded Ottoman positions in the Arabian Peninsula and took part in a blockade of the coast . Worn out by the end of the war in October 1912 , Aretusa was sold for scrap that December and broken up . \n"
3,
4,
5,= = Meteorological history = = \n
6,"The game remained scoreless in the early innings , with only two baserunners apiece for each team through the first three innings . The Giants loaded the bases with two outs in the top of the 4th inning , but Brian Johnson struck out to end the inning . The first runs of the game came in the top of the 5th as Henry Rodriguez singled to lead off the inning and scored on a home run by Gary Gaetti to give the Cubs a 2 – 0 lead . The Cubs threatened again in the next inning with singles by Lance Johnson and Sammy Sosa . Rich Rodriguez relieved Gardner , but loaded the bases with a walk to Mark Grace and allowed a single to Matt Mieske which scored two more runs and extended the Cubs ' lead to 4 – 0 . John Johnstone relieved Rodriguez and closed the inning without further scoring . \n"
7,
8,
9,"Many critics believed that the number of releases of Guitar Hero games was "" milking "" the brand name and oversaturating the market . PaRappa the Rapper creator Masaya Matsuura stated that the video game market was growing stale and needed to move beyond games that simply challenge the player to mimic the playing of licensed music . Ryan Geddes of IGN stated that he "" hit the wall with play @-@ along music games "" , and challenged the game makers to explore other ways to combine music and video games . Analysts stated that such games must continue to innovate instead of just providing more songs in order to prevent "" genre fatigue "" . Jesse Divnich of Electronic Entertainment and Design Research commented that , much like Dance Dance Revolution , Guitar Hero and other music games explosively grew initially due to significant new features from other games but have become stagnant due to focusing on content over features , and suggested that for the genre to continue to grow , they must look to incremental changes as done with the first @-@ person shooter genre . Former CEO for RedOctane , Kelly Sumner , believed that Activision "" abused "" the series , as "" they tried to get too much out of the franchise too quickly "" . \n"


見ての通り，テキストの中にはウィキペディアの記事の全段落が含まれているものもあれば，タイトルや空の行だけのものもあります。
<!-- As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines. -->

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead. For the tokenizer, you can replace the checkpoint by the one you trained yourself.

In [ ]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "sgugger/gpt2-like-tokenizer"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
tokenized_datasets["train"][1]

{'attention_mask': [1, 1, 1, 1, 1, 1],
 'input_ids': [238, 8576, 9441, 2987, 238, 252]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' the " Nameless ", a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". \n The game began development in 2010, carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series, it also underwent multiple adjustments, such as making the game more forgiving for series newcomers. Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries, along with Valkyria Chronicles II director Takeshi Ozawa. A large'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. First we create the model using the same config as our checkpoint, but initialized with random weights:

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

And we will needsome `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    f"{model_checkpoint}-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,,
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)

And we can train our model:

In [ ]:
trainer.train()

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## マスク化言語モデル
<!-- ## Masked language modeling -->

マスク化言語モデル (MLM) では，先ほどのデータセットと同じ前処理を行いますが，1 つだけ追加のステップがあります。
それは、いくつかのトークンをランダムにマスクし (`[MASK]` で置き換える)，ラベルはマスクされたトークンだけを含むように調整されます (マスクされていないトークンを予測する必要はありません)。
自分で学習したトークン化器を使う場合は `[MASK]` というトークンが，学習時に渡された特別なトークンの中にあることを確認してください。
<!-- For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). 
If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!-->

この例では [bert-base-cased`](https://huggingface.co/bert-based-cased) モデルを使用します。
代わりに [こちら](https://huggingface.co/models?filter=masked-lm) に記載されているチェックポイントのどれかを選ぶことができます。
トークン化器については，チェックポイントをあなたが学習したものに置き換えてください。
<!-- We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example. 
You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead. 
For the tokenizer, replace the checkpoint by the one you trained. -->

In [11]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

先ほどと同じトークン化機能を適用することができます。
ただ，先ほど選んだチェックポイントを使用するようにトークン化器を更新する必要があります。
<!-- We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked: -->

In [14]:
from transformers import AutoTokenizer

def tokenize_function(examples):
    return tokenizer(examples["text"])

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


先ほどと同じように，テキストをグループ化して，長さ `block_size` のサンプルにチャンクします。
データセットが個々の文で構成されている場合は，このステップを省略することができます。
<!-- And like before, we group texts together and chunk them in samples of length `block_size`. 
You can skip that step if your dataset is composed of individual sentences. -->

In [16]:
# block_size = tokenizer.model_max_length
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

あとは 2 つの例外を除いて，非常によく似ています。
まず，マスク化 LM に適したモデルを使います。
<!-- The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM: -->

In [19]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

`TrainingArguments` を再定義します。
<!-- We redefine our `TrainingArguments`: -->

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_checkpoint}-wikitext2",
)

/usr/local/lib/python3.7/dist-packages/transformers/training_args.py:926: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case asakawa/bert-base-cased-wikitext2).
  FutureWarning,


前回同様，最後の 2 つの引数は，訓練終了時にモデルを [Hub](https://huggingface.co/models) にプッシュするための設定です。
また `push_to_hub_model_id` の値をお好みのものに変更してください。
<!-- Like before, the last two arguments are to setup everything so we can push the model to the [Hub](https://huggingface.co/models) at the end of training. 
Remove the two of them if you didn't follow the installation steps at the top of the notebook, otherwise you can change the value of `push_to_hub_model_id` to something you would prefer. -->

最後に，特別な`data_collator` を使います。
`data_collator` は，サンプルを取得してテンソルでバッチ処理する役割を持つ関数です。
先ほどの例では，特に何もすることがなかったので，この引数にはデフォルトを使いました。
ここでは，ランダムマスキングを行いたいと思います。
これを (トークン化のように) 前処理として行うこともできますが，そうするとエポックごとにトークンが常に同じようにマスクされてしまいます。
このステップを `data_collator` の中で行うことで，データを確認するたびに，このランダムなマスキングが新しい方法で行われるようになります。

このマスキングを行うために，ライブラリは `DataCollatorForLanguageModeling` を提供しています。
マスキングの確率を調整することができます。
<!-- Finally, we use a special `data_collator`. 
The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. 
In the previous example, we had nothing special to do, so we just used the default for this argument. 
Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. 
By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking: -->

In [21]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

あとは，すべてを `Trainer` に渡して，訓練を開始するだけです。
<!-- Then we just have to pass everything to `Trainer` and begin training: -->

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

Cloning https://huggingface.co/asakawa/bert-base-cased-wikitext2 into local empty directory.


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 18761
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7038


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


前述のように，検証セットを用いてモデルを評価することができます。
MLM 目的では，マスクされたトークン (全体の 15％ を占める) を予測するだけで，残りのトークンを利用することができるので，パープレキシティ (錯綜度) は CLM 目的の場合よりもはるかに低くなります。
そのため，モデルにとっては簡単な課題です。
<!-- Like before, we can evaluate our model on the validation set. 
The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. 
It's thus an easier task for the model. -->

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

このデモでは，少ないデータセットで少ないエポック数の訓練を行ったため，パープレキシティはまだかなり高くなっています。
実際の LM 訓練では，より大きなデータセットとより多くのエポックが必要になるでしょう。
<!-- The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. 
For a real LM training, you  would need a larger dataset and more epochs. -->

訓練結果をハブにアップロードするには，この手順を実行します。
<!-- You can now upload the result of the training to the Hub, just execute this instruction: -->

In [ ]:
trainer.push_to_hub()

このモデルを友人や家族，お気に入りのペットと共有することができます。
例えば「あなたのユーザ名/あなたが選んだ名前」という識別子でモデルを読み込むことができます。
<!-- You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance: -->

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```